In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
from scipy import constants
from scipy.constants import physical_constants as pc
UKB = pc["Boltzmann constant"][0]
UNA = pc["Avogadro constant"][0]
eV_to_J = constants.eV

Inputs

In [3]:
T = 1e4
lev_file = "/home/zanardi/Codes/TC/HyperNet/database/levels/O3_UMN/O2.csv"

In [4]:
# Save path
out_path = "./../species"
os.makedirs(out_path, exist_ok=True)

Species
> Atom

In [5]:
atom = {
  "name": "O",
  "m": 15.9994e-3 / UNA,
  "e_f": 246795.0 / UNA,
  "lev": {
    "g": [9.0],
    "e": [0.0]
  }
}

In [6]:
with open(out_path+"/atom.json", "w") as file:
  json.dump(atom, file, indent=2)

> Molecule

In [7]:
levels = pd.read_csv(lev_file)
vqn, g, e, evib = [levels[k].to_numpy() for k in ("vqn", "g", "E", "EVib")]
# Energies
e = (e-np.amin(e)) * eV_to_J
evib = (evib-np.amin(evib)) * eV_to_J

In [9]:
def get_mapping(vqn):
  lev_to_bin = np.zeros_like(vqn)
  vmin, vmax = int(np.amin(vqn)), int(np.amax(vqn))
  for (i, vi) in enumerate(range(vmin, vmax+1)):
    lev_to_bin[vqn == vi] = i
  lev_to_bin -= np.amin(lev_to_bin)
  return lev_to_bin.astype(np.int32)

def construct_prob_mat(vqn):
  mapping = get_mapping(vqn)
  mapping -= np.amin(mapping)
  nb_lev = len(mapping)
  nb_comp = np.amax(mapping) + 1
  indices = (np.arange(nb_lev), mapping)
  P = np.zeros((nb_lev, nb_comp))
  P[indices] = 1.0
  return P

def q_int(g, e, T):
  return g * np.exp(-e/(UKB*T))

In [10]:
P = construct_prob_mat(vqn)
q = q_int(g, e, T)
Q = P.T @ q
P *= q.reshape(-1,1) / Q.reshape(1,-1)
ei = P.T @ e
gi = Q * np.exp(ei/(UKB*T))

In [11]:
molecule = {
  "name": "O2",
  "m": 31.9988e-3 / UNA,
  "e_f": 0.0,
  "e_d": np.amax(ei) * eV_to_J,
  "lev": {
    "g": gi.tolist(),
    "e": ei.tolist()
  }
}

In [12]:
with open(out_path+"/molecule.json", "w") as file:
  json.dump(molecule, file, indent=2)